In [1]:
import json
import csv
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom
import re

### Step 1: Generating xml

#### Step 1.1 Preprocess CSV file 

In [2]:

def preprocess_csv(input_file, output_file):
    """
    Processes the CSV file by mapping each of the timestep values to unique integers.

    Parameters:
    input_file (str): The path to the input CSV file. //Might update with systemn
    output_file (str): The path to the output preprocessed CSV file. //Chage this to store the csv somewhere else

    Returns:
    None
    """
    timestep_mapping = {} # Dictionary to map each timestep to a unique integer
    current_timestep = 0 
    
    with open(input_file, 'r') as file:
        csv_reader = csv.reader(file) # Read the CSV file
        rows = list(csv_reader) # Store the rows of the CSV in a list
    
    with open(output_file, 'w', newline='') as file:
        csv_writer = csv.writer(file) # Write to the output CSV file

        #Iterate through each row in the CSV file
        for row in rows:
            timestep = float(row[0]) # Extract the timestep value from the row
            
            if timestep not in timestep_mapping: # If the timestep is not already mapped
                timestep_mapping[timestep] = current_timestep # Map the timestep to the current current_timestep equal 
                current_timestep += 1  # Increment the current_timestep
            
            row[0] = str(timestep_mapping[timestep]) # Update the timestep value in the row
            csv_writer.writerow(row) 

# input and output CSV file paths
input_file = 'build/bin/output.csv'
output_file = 'preprocessed_output.csv'

# Preprocess the CSV file
preprocess_csv(input_file, output_file)
print(f"Preprocessed CSV file created: {output_file}")

Preprocessed CSV file created: preprocessed_output.csv


#### Step 1.2 Create Graphs list from Preprocessed CSV

In [ ]:
# Load the preprocessed CSV file
df = pd.read_csv('preprocessed_output.csv', header=None) #Header row doesnt exists
df.columns = ['Timestep', 'chunkID', 'source', 'destination']

# print(df.head) #Print the first 5 rows of the dataframe. Use for debugging

# Assuming height and width of the mesh are known
height = 3  # Adjust as necessary based on mesh structure
width = 3   # Adjust as necessary based on mesh structure

def add_all_nodes(G, height, width):
    """
    Adds all nodes to the graph G.

    Parameters:
    - G (networkx.Graph): The graph to which the nodes will be added.
    - height (int): The height of the grid.
    - width (int): The width of the grid.

    Returns:
    None
    """
    for i in range(height * width):  # Assuming node identifiers are from 0 to height*width-1
        G.add_node(i) #in all the mesh, add nodes

# Get unique timesteps
timesteps = df['Timestep'].unique()

graphs = []

for timestep in timesteps:
    timestep_data = df[df['Timestep'] == timestep] # slice the dataframes into t = 0, 1...N
    print(timestep_data.head)
    
    G = nx.DiGraph() #Create a directed graph
    add_all_nodes(G, height, width)  # Add nodes for this timestep
    
    # Directly use integer identifiers from the CSV
    for _, row in timestep_data.iterrows(): #Iterate through the rows of the dataframe
        source = row['source'] #Extract the source node
        print('src = ',source) 
        destination = row['destination'] #Extract the destination node
        print('dst = ', destination)
        chunkID = row['chunkID']    #Extract the chunkID
        print('chunk = ', chunkID)
        G.add_edge(source, destination, chunkID=chunkID) #Add these nodes as the edge to the graph
    
    graphs.append(G) #Append this graph to the list of graphs

####  Step 1.3 Create data structure node_traffic from graphs list

In [20]:
# Normally you would load data from a file like so:
# df = pd.read_csv("path_to_your_file.csv")
# Load the preprocessed CSV file
df = pd.read_csv('preprocessed_output.csv', header=None) #Header row doesnt exists
df.columns = ['Timestep', 'chunkID', 'source', 'destination']

# Create graphs per timestep
graphs = {t: nx.DiGraph() for t in df['Timestep'].unique()}

# Populate each graph
for _, row in df.iterrows():
    graphs[row['Timestep']].add_edge(row['source'], row['destination'], chunkID=row['chunkID'])

# Initialize node_traffic from all possible edges in the first graph
node_traffic = {}
first_graph = graphs[0]  # Assuming the first timestep graph is fully representative
for node in first_graph.nodes():
    node_traffic[node] = {}
    edge_counter = 0  # Reset counter for each node
    # Add thread blocks for all outgoing edges
    for target in first_graph.successors(node):
        node_traffic[node][(node, target)] = {
            'threadID': edge_counter,
            'records': [{'timestep': t, 'chunkID': -1, 'type': '-1'} for t in graphs.keys()]
        }
        edge_counter += 1
    # Add thread blocks for all incoming edges if not already added
    for source in first_graph.predecessors(node):
        if (source, node) not in node_traffic[node]:
            node_traffic[node][(source, node)] = {
                'threadID': edge_counter,
                'records': [{'timestep': t, 'chunkID': -1, 'type': '-1'} for t in graphs.keys()]
            }
            edge_counter += 1

# Populate the node_traffic with actual data from all timestep graphs
for t_index, graph in graphs.items():
    for source, destination, data in graph.edges(data=True):
        node_traffic[source][(source, destination)]['records'][t_index]['chunkID'] = data['chunkID']
        node_traffic[source][(source, destination)]['records'][t_index]['type'] = 's'
        node_traffic[destination][(source, destination)]['records'][t_index]['chunkID'] = data['chunkID']
        node_traffic[destination][(source, destination)]['records'][t_index]['type'] = 'r'

# Debug: Print out node_traffic for node 0 to see thread blocks
print("Node 0 Thread Blocks:")
for edge_key, tb_info in node_traffic[0].items():
    print(f"Edge {edge_key} with Thread ID {tb_info['threadID']}:")
    for record in tb_info['records']:
        print(record)

Node 0 Thread Blocks:
Edge (0, 1) with Thread ID 0:
{'timestep': 0, 'chunkID': 0, 'type': 's'}
{'timestep': 1, 'chunkID': 3, 'type': 's'}
{'timestep': 2, 'chunkID': 6, 'type': 's'}
{'timestep': 3, 'chunkID': -1, 'type': '-1'}
{'timestep': 4, 'chunkID': -1, 'type': '-1'}
Edge (0, 3) with Thread ID 1:
{'timestep': 0, 'chunkID': 0, 'type': 's'}
{'timestep': 1, 'chunkID': 1, 'type': 's'}
{'timestep': 2, 'chunkID': -1, 'type': '-1'}
{'timestep': 3, 'chunkID': 2, 'type': 's'}
{'timestep': 4, 'chunkID': -1, 'type': '-1'}
Edge (1, 0) with Thread ID 2:
{'timestep': 0, 'chunkID': 1, 'type': 'r'}
{'timestep': 1, 'chunkID': 4, 'type': 'r'}
{'timestep': 2, 'chunkID': 2, 'type': 'r'}
{'timestep': 3, 'chunkID': 5, 'type': 'r'}
{'timestep': 4, 'chunkID': -1, 'type': '-1'}
Edge (3, 0) with Thread ID 3:
{'timestep': 0, 'chunkID': 3, 'type': 'r'}
{'timestep': 1, 'chunkID': 6, 'type': 'r'}
{'timestep': 2, 'chunkID': 7, 'type': 'r'}
{'timestep': 3, 'chunkID': 8, 'type': 'r'}
{'timestep': 4, 'chunkID': -1, 

In [ ]:
from pprint import pprint
pprint(node_traffic)

In [ ]:
def pretty_print_node_traffic(data, indent=0):
    for node, edges in data.items():
        print('    ' * indent + str(node) + ':')
        if isinstance(edges, dict):
            pretty_print_node_traffic(edges, indent + 1)
        else:
            print('    ' * (indent + 1) + str(edges))

pretty_print_node_traffic(node_traffic)


#### Step 1.4 Generate XML from data structure node_traffic & dump in XML file

In [23]:
def parse_output_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        # Using regular expressions to find the desired pattern
        pattern = r'\[All-Reduce Information\]\s*#Chunks/NPU:\s*(\d+)'
        match = re.search(pattern, content)
        if match:
            # Extracting the value of #Chunks/NPU
            chunks_per_npu = int(match.group(1))
            return chunks_per_npu
        else:
            # If the pattern is not found
            return None

# Example usage:
file_path = "./TACOS_output.txt" # path to the file (from ./run.sh)
chunks_per_npu = parse_output_file(file_path)
if chunks_per_npu is not None:
    print("Chunks/NPU:", chunks_per_npu)

# Function to generate and save XML in a pretty format
def generate_and_save_xml_pretty(node_traffic, filename):
    root = ET.Element("network")
    for node, edges in node_traffic.items():
        gpu_elem = ET.SubElement(root, "gpu", id=str(node))
        for (source, destination), edge_info in edges.items():
            tb_elem = ET.SubElement(gpu_elem, "tb", id=str(edge_info['threadID']),
                                    send=str(destination if edge_info['records'][0]['type'] == 's' else '-1'),
                                    recv=str(source if edge_info['records'][0]['type'] == 'r' else '-1'))
            for record in edge_info['records']:
                ET.SubElement(tb_elem, "step", s=str(record['timestep']), type=record['type'], chunk=str(record['chunkID']))

    # Convert to string using ElementTree and parse with minidom for pretty printing
    rough_string = ET.tostring(root, 'utf-8')
    reparsed = xml.dom.minidom.parseString(rough_string)
    pretty_xml_as_string = reparsed.toprettyxml(indent="  ")

    # Write to file
    with open(filename, 'w') as file:
        file.write(pretty_xml_as_string)

Chunks/NPU: 1


In [24]:
# Usage
filename = 'network_flow_pretty.xml'
generate_and_save_xml_pretty(node_traffic, filename)
print(f"XML file saved as {filename}")

XML file saved as network_flow_pretty.xml
